<a href="https://www.kaggle.com/code/rutikachavan/housing-project?scriptVersionId=103589646" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
A=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")

In [3]:
A.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
A.shape

(1460, 81)

In [5]:
A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# Missing Data Treatment

In [6]:
A.Alley = A.Alley.fillna("No alley access")
A.BsmtQual = A.BsmtQual.fillna("No Basement")
A.BsmtCond = A.BsmtCond.fillna("No Basement")
A.BsmtExposure = A.BsmtExposure.fillna("No Basement")
A.BsmtFinType1 = A.BsmtFinType1.fillna("No Basement")
A.BsmtFinType2 = A.BsmtFinType2.fillna("No Basement")
A.FireplaceQu = A.FireplaceQu.fillna("No Fireplace")
A.GarageType = A.GarageType.fillna("No Garage")
A.GarageFinish = A.GarageFinish.fillna("No Garage")
A.GarageQual = A.GarageQual.fillna("No Garage")
A.GarageCond = A.GarageCond.fillna("No Garage")
A.PoolQC = A.PoolQC.fillna("No Pool")
A.Fence = A.Fence.fillna("No Fence")
A.MiscFeature = A.MiscFeature.fillna("None")

In [7]:
A.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [8]:
for i in A.columns:
    if(A[i].dtypes=="object"):
        x=A[i].mode()[0]
        A[i]=A[i].fillna(x)
    else:
        x=A[i].mean()
        A[i]=A[i].fillna(x)

In [9]:
A.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

# EDA

In [10]:
cat=[]
con=[]
for i in A.columns:
    if A[i].dtypes=="object":
        cat.append(i)
    else:
        con.append(i)

In [11]:
len(cat)

43

In [12]:
Q=A[con].corr()["SalePrice"].sort_values()

In [13]:
imp_con_col=list(Q[(Q<1)&(Q>0.4)].index)

In [14]:
imp_con_col

['Fireplaces',
 'GarageYrBlt',
 'MasVnrArea',
 'YearRemodAdd',
 'YearBuilt',
 'TotRmsAbvGrd',
 'FullBath',
 '1stFlrSF',
 'TotalBsmtSF',
 'GarageArea',
 'GarageCars',
 'GrLivArea',
 'OverallQual']

In [15]:
def ANOVA(df,con_col):
    cat=[ ]
    con=[ ]
    for i in df.columns:
        if df[i].dtypes=="object":
            cat.append(i)
        else:
            con.append(i)
            
    for i in cat:
        from statsmodels.formula.api import ols
        relation = con_col + " ~ " + i
        model = ols(relation,df).fit()
        from statsmodels.stats.anova import anova_lm
        anova_results = anova_lm(model)
            
        print("---------SalePrice vs",i,"---------------")
        print(anova_results.iloc[0,4])

In [16]:
ANOVA(A,"SalePrice")

---------SalePrice vs MSZoning ---------------
8.817633866276532e-35
---------SalePrice vs Street ---------------
0.11704860406782483
---------SalePrice vs Alley ---------------
2.996379680546275e-07
---------SalePrice vs LotShape ---------------
6.447523852013415e-25
---------SalePrice vs LandContour ---------------
2.7422167521379096e-08
---------SalePrice vs Utilities ---------------
0.5847167739689381
---------SalePrice vs LotConfig ---------------
3.1631674736044365e-06
---------SalePrice vs LandSlope ---------------
0.1413963584114357
---------SalePrice vs Neighborhood ---------------
1.5586002827702678e-225
---------SalePrice vs Condition1 ---------------
8.904549416139423e-08
---------SalePrice vs Condition2 ---------------
0.043425658360948464
---------SalePrice vs BldgType ---------------
2.0567364604967015e-10
---------SalePrice vs HouseStyle ---------------
3.376776535121222e-25
---------SalePrice vs RoofStyle ---------------
3.653523047100657e-17
---------SalePrice vs Roof

In [17]:
imp_cat_cols = ["MSZoning","Street","Alley","LotShape","Utilities","Neighborhood","Condition1","Condition2","HouseStyle","RoofMatl","Exterior1st","ExterQual","Foundation","Heating","FireplaceQu","PoolQC"]

# Skewness

In [18]:
Q=A.skew()

In [19]:
Q

Id                0.000000
MSSubClass        1.407657
LotFrontage       2.384950
LotArea          12.207688
OverallQual       0.216944
OverallCond       0.693067
YearBuilt        -0.613461
YearRemodAdd     -0.503562
MasVnrArea        2.676412
BsmtFinSF1        1.685503
BsmtFinSF2        4.255261
BsmtUnfSF         0.920268
TotalBsmtSF       1.524255
1stFlrSF          1.376757
2ndFlrSF          0.813030
LowQualFinSF      9.011341
GrLivArea         1.366560
BsmtFullBath      0.596067
BsmtHalfBath      4.103403
FullBath          0.036562
HalfBath          0.675897
BedroomAbvGr      0.211790
KitchenAbvGr      4.488397
TotRmsAbvGrd      0.676341
Fireplaces        0.649565
GarageYrBlt      -0.668175
GarageCars       -0.342549
GarageArea        0.179981
WoodDeckSF        1.541376
OpenPorchSF       2.364342
EnclosedPorch     3.089872
3SsnPorch        10.304342
ScreenPorch       4.122214
PoolArea         14.828374
MiscVal          24.476794
MoSold            0.212053
YrSold            0.096269
S

In [20]:
col_with_skew=list(Q[Q>2].index)

In [21]:
col_with_skew

['LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinSF2',
 'LowQualFinSF',
 'BsmtHalfBath',
 'KitchenAbvGr',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [22]:
import numpy as np

In [23]:
#for j in col_with_skew:
    #w=[]
    #for i in A[j]:
        #if i!=0:
            #w.append(np.log(i))
        #else:
            #w.append(i)
        #A[j]=w
    
for j in col_with_skew:
    W = []
    for i in A[j]:
        if(i != 0):
            W.append(np.log(i))
        else:
            W.append(i)
    A[j] = W

In [24]:
A[col_with_skew].skew()


LotFrontage      -0.930795
LotArea          -0.137994
MasVnrArea        0.483252
BsmtFinSF2        2.524777
LowQualFinSF      7.462312
BsmtHalfBath     26.990701
KitchenAbvGr      4.441196
OpenPorchSF      -0.016306
EnclosedPorch     2.113838
3SsnPorch         7.737554
ScreenPorch       3.150990
PoolArea         14.363144
MiscVal           5.171227
dtype: float64

In [25]:
B=A[imp_con_col].join(A[imp_cat_cols])

# Preprocessing

In [26]:
def preprocessing(df):
    import pandas as pd
    cat = []
    con = []
    for i in df.columns:
        if(df[i].dtypes == "object"):
            cat.append(i)
        else:
            con.append(i)
    X1 = pd.get_dummies(df[cat])
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X2 = pd.DataFrame(ss.fit_transform(df[con]),columns=con)
    X3 = X2.join(X1)
    return X3
Xnew=preprocessing(B)

# Define X & Y

In [27]:
Y=A[["SalePrice"]]
X=Xnew

In [28]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=21)

# Backward Elimination

In [29]:
from statsmodels.api import OLS,add_constant
xconst=add_constant(xtrain)
ols=OLS(ytrain,xconst)
model=ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     73.43
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        07:00:17   Log-Likelihood:                -13554.
No. Observations:                1168   AIC:                         2.733e+04
Df Residuals:                    1055   BIC:                         2.791e+04
Df Model:                         112                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     7.367e+16   7.75e+16      0.951      0.342   -7.83e+16    2.26e+17
Fireplaces                9059.7847   2216.534      4.087      0.000    4710.469    1.34e+04
GarageYrBlt               1054.0412   1634.160      0.645      0.519   -2152.532    4260.615
MasVnrArea                1513.8667   1124.427      1.346      0.178    -692.500    3720.233
YearRemodAdd              6375.9635   1274.890      5.001      0.000    3874.355    8877.572
YearBuilt                 6804.4303   2580.632      2.637      0.008    1740.674    1.19e+04
TotRmsAbvGrd             -1253.0873   1732.365     -0.723      0.470   -4652.359    2146.185
FullBath                 -3198.6443   1398.166     -2.288      0.022   -5942.147    -455.142
1stFlrSF                 -9558.4794   3113.007     -3.070      0.002   -1.57e+04   -3450.091
TotalBsmtSF               1.536e+04   2194.913      6.998      0.000    1.11e+04    1.97e+04
GarageArea                6304.3016   2049.789      3.076      0.002    2282.175    1.03e+04
GarageCars                1341.9637   2107.215      0.637      0.524   -2792.846    5476.774
GrLivArea                 4.088e+04   3528.289     11.587      0.000     3.4e+04    4.78e+04
OverallQual                1.38e+04   1705.087      8.095      0.000    1.05e+04    1.71e+04
MSZoning_C (all)         -7.356e+15   7.74e+15     -0.951      0.342   -2.25e+16    7.82e+15
MSZoning_FV              -7.356e+15   7.74e+15     -0.951      0.342   -2.25e+16    7.82e+15
MSZoning_RH              -7.356e+15   7.74e+15     -0.951      0.342   -2.25e+16    7.82e+15
MSZoning_RL              -7.356e+15   7.74e+15     -0.951      0.342   -2.25e+16    7.82e+15
MSZoning_RM              -7.356e+15   7.74e+15     -0.951      0.342   -2.25e+16    7.82e+15
Street_Grvl              -3.362e+16   3.53e+16     -0.951      0.342   -1.03e+17    3.57e+16
Street_Pave              -3.362e+16   3.53e+16     -0.951      0.342   -1.03e+17    3.57e+16
Alley_Grvl               -2.416e+16   2.54e+16     -0.951      0.342    -7.4e+16    2.57e+16
Alley_No alley access    -2.416e+16   2.54e+16     -0.951      0.342    -7.4e+16    2.57e+16
Alley_Pave               -2.416e+16   2.54e+16     -0.951      0.342    -7.4e+16    2.57e+16
LotShape_IR1              6.866e+15   7.22e+15      0.951      0.342    -7.3e+15     2.1e+16
LotShape_IR2              6.866e+15   7.22e+15      0.951      0.342    -7.3e+15     2.1e+16
LotShape_IR3              6.866e+15   7.22e+15      0.951      0.342    -7.3e+15     2.1e+16
LotShape_Reg              6.866e+15   7.22e+15      0.951      0.342    -7.3e+15     2.1e+16
Utilities_AllPub         -3.196e+16   3.36e+16     -0.951      0.342   -9.79e+16     3.4e+16
Utilities_NoSeWa         -3.196e+16   3.36e+16     -0.951      0.342   -9.79e+16     3.4e+16
Neighborhood_B

In [30]:
p=model.pvalues

In [31]:
q=model.pvalues.max()

In [32]:
col_to_drop=list(p[p==q].index)[0]

In [33]:
Xnew=X.drop(labels=[col_to_drop],axis=1)

In [34]:
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

In [35]:
from statsmodels.api import OLS,add_constant
xconst=add_constant(xtrain)
ols=OLS(ytrain,xconst)
model=ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     74.18
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        07:00:18   Log-Likelihood:                -13554.
No. Observations:                1168   AIC:                         2.733e+04
Df Residuals:                    1056   BIC:                         2.790e+04
Df Model:                         111                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     6.667e+16   7.75e+16      0.861      0.390   -8.53e+16    2.19e+17
Fireplaces                9082.3582   2215.476      4.100      0.000    4735.122    1.34e+04
GarageYrBlt                909.1794   1619.442      0.561      0.575   -2268.510    4086.869
MasVnrArea                1565.7724   1120.394      1.398      0.163    -632.680    3764.225
YearRemodAdd              6419.3450   1273.877      5.039      0.000    3919.726    8918.964
YearBuilt                 6993.3084   2563.481      2.728      0.006    1963.212     1.2e+04
TotRmsAbvGrd             -1228.5460   1729.684     -0.710      0.478   -4622.555    2165.463
FullBath                 -3198.5753   1397.038     -2.290      0.022   -5939.862    -457.289
1stFlrSF                 -9563.5817   3110.762     -3.074      0.002   -1.57e+04   -3459.604
TotalBsmtSF               1.538e+04   2189.781      7.022      0.000    1.11e+04    1.97e+04
GarageArea                7364.3955   1197.701      6.149      0.000    5014.251    9714.540
GrLivArea                  4.09e+04   3526.019     11.600      0.000     3.4e+04    4.78e+04
OverallQual               1.379e+04   1706.405      8.082      0.000    1.04e+04    1.71e+04
MSZoning_C (all)         -5.122e+15   5.95e+15     -0.861      0.390   -1.68e+16    6.56e+15
MSZoning_FV              -5.122e+15   5.95e+15     -0.861      0.390   -1.68e+16    6.56e+15
MSZoning_RH              -5.122e+15   5.95e+15     -0.861      0.390   -1.68e+16    6.56e+15
MSZoning_RL              -5.122e+15   5.95e+15     -0.861      0.390   -1.68e+16    6.56e+15
MSZoning_RM              -5.122e+15   5.95e+15     -0.861      0.390   -1.68e+16    6.56e+15
Street_Grvl              -2.682e+16   3.12e+16     -0.861      0.390    -8.8e+16    3.43e+16
Street_Pave              -2.682e+16   3.12e+16     -0.861      0.390    -8.8e+16    3.43e+16
Alley_Grvl               -1.899e+16   2.21e+16     -0.861      0.390   -6.23e+16    2.43e+16
Alley_No alley access    -1.899e+16   2.21e+16     -0.861      0.390   -6.23e+16    2.43e+16
Alley_Pave               -1.899e+16   2.21e+16     -0.861      0.390   -6.23e+16    2.43e+16
LotShape_IR1              7.594e+15   8.82e+15      0.861      0.390   -9.72e+15    2.49e+16
LotShape_IR2              7.594e+15   8.82e+15      0.861      0.390   -9.72e+15    2.49e+16
LotShape_IR3              7.594e+15   8.82e+15      0.861      0.390   -9.72e+15    2.49e+16
LotShape_Reg              7.594e+15   8.82e+15      0.861      0.390   -9.72e+15    2.49e+16
Utilities_AllPub         -2.534e+16   2.94e+16     -0.861      0.390   -8.31e+16    3.24e+16
Utilities_NoSeWa         -2.534e+16   2.94e+16     -0.861      0.390   -8.31e+16    3.24e+16
Neighborhood_Blmngtn      2.732e+15   3.17e+15      0.861      0.390    -3.5e+15    8.96e+15
Neighborhood_B

In [36]:
p = model.pvalues
q = model.pvalues.max()
col_to_drop = list(p[p == q].index)[0]
Xnew = Xnew.drop(labels = [col_to_drop],axis=1)
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)
from statsmodels.api import OLS,add_constant
xconst = add_constant(xtrain)
ols = OLS(ytrain,xconst)
model = ols.fit()

In [37]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     74.89
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        07:00:18   Log-Likelihood:                -13555.
No. Observations:                1168   AIC:                         2.733e+04
Df Residuals:                    1057   BIC:                         2.789e+04
Df Model:                         110                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     6.319e+16   7.71e+16      0.820      0.412    -8.8e+16    2.14e+17
Fireplaces                9047.8438   2213.173      4.088      0.000    4705.131    1.34e+04
MasVnrArea                1588.4255   1120.206      1.418      0.156    -609.654    3786.505
YearRemodAdd              6543.6201   1254.534      5.216      0.000    4081.959    9005.281
YearBuilt                 7387.7797   2462.359      3.000      0.003    2556.111    1.22e+04
TotRmsAbvGrd             -1255.0106   1728.308     -0.726      0.468   -4646.315    2136.294
FullBath                 -3130.3454   1391.488     -2.250      0.025   -5860.739    -399.952
1stFlrSF                 -9505.1802   3107.595     -3.059      0.002   -1.56e+04   -3407.424
TotalBsmtSF               1.531e+04   2187.831      6.997      0.000     1.1e+04    1.96e+04
GarageArea                7481.7641   1180.628      6.337      0.000    5165.123    9798.406
GrLivArea                 4.091e+04   3524.951     11.605      0.000     3.4e+04    4.78e+04
OverallQual                1.38e+04   1703.383      8.102      0.000    1.05e+04    1.71e+04
MSZoning_C (all)         -4.849e+15   5.91e+15     -0.820      0.412   -1.65e+16    6.76e+15
MSZoning_FV              -4.849e+15   5.91e+15     -0.820      0.412   -1.65e+16    6.76e+15
MSZoning_RH              -4.849e+15   5.91e+15     -0.820      0.412   -1.65e+16    6.76e+15
MSZoning_RL              -4.849e+15   5.91e+15     -0.820      0.412   -1.65e+16    6.76e+15
MSZoning_RM              -4.849e+15   5.91e+15     -0.820      0.412   -1.65e+16    6.76e+15
Street_Grvl              -2.538e+16    3.1e+16     -0.820      0.412   -8.61e+16    3.54e+16
Street_Pave              -2.538e+16    3.1e+16     -0.820      0.412   -8.61e+16    3.54e+16
Alley_Grvl                -1.82e+16   2.22e+16     -0.820      0.412   -6.18e+16    2.54e+16
Alley_No alley access     -1.82e+16   2.22e+16     -0.820      0.412   -6.18e+16    2.54e+16
Alley_Pave                -1.82e+16   2.22e+16     -0.820      0.412   -6.18e+16    2.54e+16
LotShape_IR1              6.888e+15    8.4e+15      0.820      0.412    -9.6e+15    2.34e+16
LotShape_IR2              6.888e+15    8.4e+15      0.820      0.412    -9.6e+15    2.34e+16
LotShape_IR3              6.888e+15    8.4e+15      0.820      0.412    -9.6e+15    2.34e+16
LotShape_Reg              6.888e+15    8.4e+15      0.820      0.412    -9.6e+15    2.34e+16
Utilities_AllPub         -2.389e+16   2.91e+16     -0.820      0.412   -8.11e+16    3.33e+16
Utilities_NoSeWa         -2.389e+16   2.91e+16     -0.820      0.412   -8.11e+16    3.33e+16
Neighborhood_Blmngtn      2.592e+15   3.16e+15      0.820      0.412   -3.61e+15     8.8e+15
Neighborhood_Blueste      2.592e+15   3.16e+15      0.820      0.412   -3.61e+15     8.8e+15
Neighborhood_B

In [38]:
Xnew.shape

(1460, 126)

# prediction on testing Data

In [39]:
C=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [40]:
C

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [41]:
C.Alley = C.Alley.fillna("No alley access")
C.BsmtQual = C.BsmtQual.fillna("No Basement")
C.BsmtCond = C.BsmtCond.fillna("No Basement")
C.BsmtExposure = C.BsmtExposure.fillna("No Basement")
C.BsmtFinType1 = C.BsmtFinType1.fillna("No Basement")
C.BsmtFinType2 = C.BsmtFinType2.fillna("No Basement")
C.FireplaceQu = C.FireplaceQu.fillna("No Fireplace")
C.GarageType = C.GarageType.fillna("No Garage")
C.GarageFinish = C.GarageFinish.fillna("No Garage")
C.GarageQual = C.GarageQual.fillna("No Garage")
C.GarageCond = C.GarageCond.fillna("No Garage")
C.PoolQC = C.PoolQC.fillna("No Pool")
C.Fence = C.Fence.fillna("No Fence")
C.MiscFeature = C.MiscFeature.fillna("None")

In [42]:
def replacer(df):
    import pandas as pd
    Q = pd.DataFrame(df.isna().sum(),columns=["ct"])
    for i in Q[Q.ct > 0].index:
        if(df[i].dtypes == "object"):
            x = df[i].mode()[0]
            df[i] = df[i].fillna(x)
        else:
            x = df[i].mean()
            df[i] = df[i].fillna(x)
replacer(C)

In [43]:
abc=preprocessing(C)

In [44]:
Q=list(Xnew.columns)
for i in ['PoolQC_Fa', 'Heating_Floor', 'RoofMatl_ClyTile', 'HouseStyle_2.5Fin', 'Condition2_RRAe', 'Condition2_RRNn', 'Exterior1st_Stone', 'Condition2_RRAn', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 'Heating_OthW', 'RoofMatl_Metal', 'Utilities_NoSeWa', 'RoofMatl_Membran']:
    abc[i]=0

In [45]:
final=abc[Q]

In [46]:
final.shape

(1459, 126)

In [47]:
Xnew.shape

(1460, 126)

In [48]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(Xnew,Y)
predicted = model.predict(final)

In [49]:
T = C[["Id"]]
T["SalePrice"] = predicted

In [50]:
T.to_csv("C:\\Users\\shree\\Desktop\\DataScience\\Day 21, Doubt session+mini project\\Practice of housing project.csv",index=None)